# 🚦 Traffic Manager AI (V11 - Cloudflare Debug)
### Full logging enabled for debugging

**Setup:**
1. Chạy Cloudflare trên máy local: `./cloudflared tunnel --url https://localhost:3000 --no-tls-verify`
2. Copy URL `.trycloudflare.com` vào `SERVER_URL` bên dưới
3. Restart Node.js server (để apply ping config mới)
4. Chạy tất cả cells

In [ ]:
# Cell 1: Dependencies + Model
!pip install ultralytics python-socketio[client] websocket-client opencv-python-headless --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11m.pt

import torch
from ultralytics import YOLO
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Loading YOLO on {device}...")
model = YOLO('yolo11m.pt').to(device)
print("✅ Model loaded!")

In [ ]:
# Cell 2: Main Detection Loop (với DEBUG LOG đầy đủ)
import cv2, numpy as np, socketio, base64, time, threading, queue, logging
from datetime import datetime

# ========== CẬP NHẬT URL CLOUDFLARE ==========
SERVER_URL = 'https://discs-lone-part-dakota.trycloudflare.com'
# =============================================

CONFIDENCE = 0.5
CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

# Enable Socket.IO debug logging
logging.basicConfig(level=logging.DEBUG)
sio_logger = logging.getLogger('socketio')
sio_logger.setLevel(logging.DEBUG)
eio_logger = logging.getLogger('engineio')
eio_logger.setLevel(logging.DEBUG)

# Socket.IO client với debug
sio = socketio.Client(
    reconnection=True,
    reconnection_attempts=0,  # Infinite
    reconnection_delay=1,
    reconnection_delay_max=5,
    ssl_verify=False,
    logger=True,
    engineio_logger=True
)

queues = {}
running = True
stats = {'frames': 0, 'detections': 0, 'connects': 0, 'disconnects': 0}

def log(msg):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def process(cid):
    log(f"📹 Camera thread started: {cid[-8:]}")
    while running:
        try:
            f, c, i, t = queues[cid].get(timeout=0.1)
            stats['frames'] += 1
            h, w = f.shape[:2]
            dets = []
            
            start = time.time()
            for r in model(f, verbose=False, conf=CONFIDENCE):
                for b in r.boxes:
                    cls = model.names[int(b.cls[0])]
                    if cls not in CLASSES: continue
                    x1,y1,x2,y2 = map(int, b.xyxy[0])
                    dets.append({'type':'vehicle','class':cls,'confidence':float(b.conf[0]),
                        'bbox':{'x1':x1/w,'y1':y1/h,'x2':x2/w,'y2':y2/h}})
            ms = (time.time() - start) * 1000
            
            if dets:
                stats['detections'] += len(dets)
                if sio.connected:
                    sio.emit('car_detected', {
                        'camera_id': c, 'image_id': i,
                        'detections': dets, 'inference_time': ms, 'created_at': t
                    })
                    log(f"✅ {c[-4:]}: {len(dets)} objs | {ms:.0f}ms | Total: {stats['frames']}F/{stats['detections']}D")
                else:
                    log(f"⚠️ Not connected, dropping {len(dets)} detections")
            
            time.sleep(0.05)  # 20 FPS max
        except queue.Empty:
            pass
        except Exception as e:
            log(f"❌ Process error: {e}")

@sio.event
def connect():
    stats['connects'] += 1
    log(f"✅ CONNECTED! (Total connects: {stats['connects']})")
    sio.emit('join_all_camera')
    log("📡 Sent join_all_camera")

@sio.event
def connect_error(data):
    log(f"❌ CONNECTION ERROR: {data}")

@sio.event
def disconnect():
    stats['disconnects'] += 1
    log(f"⚠️ DISCONNECTED! (Total disconnects: {stats['disconnects']})")

@sio.on('image')
def on_image(d):
    try:
        img = d.get('image') or d.get('buffer')
        if isinstance(img, dict): img = bytes(img.get('data', []))
        if isinstance(img, str): img = base64.b64decode(img)
        f = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)
        if f is None:
            log("⚠️ Failed to decode image")
            return
        
        c = d['cameraId']
        if c not in queues:
            queues[c] = queue.Queue(maxsize=5)
            threading.Thread(target=process, args=(c,), daemon=True).start()
            log(f"📷 New camera: {c[-8:]}")
        
        q = queues[c]
        if q.full():
            q.get_nowait()  # Drop oldest
        q.put((f, c, d['imageId'], d.get('created_at', 0)))
    except Exception as e:
        log(f"❌ Image handler error: {e}")

def maintain():
    while running:
        if not sio.connected:
            log(f"🔄 Connecting to {SERVER_URL}...")
            try:
                sio.connect(SERVER_URL, transports=['websocket', 'polling'])
            except Exception as e:
                log(f"❌ Connect failed: {e}")
                time.sleep(2)
        time.sleep(1)

# Start
log(f"🎯 Target: {SERVER_URL}")
log(f"📊 Stats tracking enabled")
threading.Thread(target=maintain, daemon=True).start()

try:
    while running:
        time.sleep(30)
        log(f"📊 Stats: {stats['frames']}F | {stats['detections']}D | {stats['connects']}C | {stats['disconnects']}DC")
except KeyboardInterrupt:
    log("🛑 Stopped by user")
    running = False